Bonus Lecture #1: Doing Data Science with <a href="https://plot.ly/">plotly</a>
=================

### Exploring the US election

In this lecture, we are going to explore some ways of visualizing data using plotly, a very cool visualization api which allows you to create professional-looking visualizations. We will do that in the context of Presidential election! We are going to combine data from various sources, just as you would do in real life data science! The data that we are using is a processed version of those found in the below sources.

**US Election results** can be found <a href="http://www.stat.berkeley.edu/users/nolan/data/voteProject/2016_US_County_Level_Presidential_Results.csv">here</a>.

**Median Income data by state** can be found here <a href="https://www.census.gov/prod/2012pubs/acsbr11-02.pdf">here</a>.

**Gini coefficient** data lives <a href="http://www.census.gov/topics/income-poverty/income-inequality.html">here</a>.

In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
from datascience import *
import numpy as np
import math

py.sign_in('oikobill', 'nebugeZ772mlIhUIxTPh')

## Election Resuts Overview 

In [2]:
#First off let's read in our data and take a quick look
#One easy way to read in your data from a csv file is using the datascience package's table
#You will learn more about the datascience package next week!
df = Table.read_table("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/election_df_2016.csv").drop("code")
df.show(5)

state_abbr,votes_dem.16,votes_gop.16,perc_diff,text
AK,93003,130413,-0.167445,AKDemocrats: 41.6277258567%Republicans: 58.37227 ...
AL,718084,1.30692e+06,-0.290784,ALDemocrats: 35.4607806681%Republicans: 64.53921 ...
AR,378729,677904,-0.28314,ARDemocrats: 35.8430031998%Republicans: 64.15699 ...
AZ,936250,1.02115e+06,-0.0433758,AZDemocrats: 47.8312090912%Republicans: 52.16879 ...
CA,5.93128e+06,3.18472e+06,0.30129,CADemocrats: 65.0645063341%Republicans: 34.93549 ...


In [3]:
#The first step is to extract the data we are interested in from our table
#all the statements below return the corresponding table columns in the form of a numpy array.
state_abbreviations = df.column("state_abbr").astype(str)
text_16 = df.column("text").astype(str)
perc_difference_16 = df.column("perc_diff").astype(str)
votes_dem_16 = df.column("votes_dem.16").astype(int)
votes_gop_16 = df.column("votes_gop.16").astype(int)
df_ = Table.read_table("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/election_df_2012.csv")
perc_difference_12 = df_.column("perc_diff").astype(str)

In [4]:
#Determine the color scale that we will be using
scl = [[0, 'rgb(255, 0, 0)'],[0.25, 'rgb(255, 122, 122)'],[0.4, "rgb(247, 133, 133)"],[0.45, 'rgb(255, 153, 153)'],
       [0.5, 'rgb(255, 255, 255)'],[0.55, 'rgb(160, 160, 255)'],[0.6, 'rgb(133, 177, 247)'], [0.75, 'rgb(122,122,255)'],[1, 'rgb(0, 0, 255)']]

#first off we have to put the data in the right format
data = [ dict(
        type='choropleth',
        autocolorscale = False,
        locations = state_abbreviations, #the state abbreviations go here
        z = perc_difference_16.astype(float),
        locationmode = 'USA-states', #determine the type of map you want to make
        text = text_16, #this is the array containing the information we will see when we hover over our states
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Percentage Difference"), #Title of our map
            colorscale = scl, #set the colorscale we created
            zmin = -0.5,
            zmax = 0.5
        ) ]

#make the layout that contains information about the map
layout = dict(
        title = '2016 Election Results',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

In [5]:
#Now I want to calculate the percentage that each party got on each state - numpy math
perc_dem = votes_dem_16/(votes_dem_16+votes_gop_16)
perc_gop = votes_gop_16/(votes_dem_16+votes_gop_16)

In [6]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    y=state_abbreviations,
    x=perc_dem,
    name='Democrats',
    orientation = 'h',
    marker = dict(
        color = 'rgba(169, 228, 232, 0.4)',
        line = dict(
            color = 'rgba(169, 228, 232, 1.0)',
            width = 5)
    )
)
trace2 = go.Bar(
    y=state_abbreviations,
    x=perc_gop,
    name='Republicans',
    orientation = 'h',
    marker = dict(
        color = 'rgba(224, 92, 45, 0.6)',
        line = dict(
            color = 'rgba(224, 92, 45, 1.0)',
            width = 5)
    )
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='marker-h-bar')

## In how many states did each party improve their performance?

This election was admittedly one of the most polarized of the recent years. So one question to ask is how did that affect the electoral map?

In [7]:
#explore in how many states republicans and democrats improved their percentage
perc_difference_12 = perc_difference_12.astype(float)
perc_difference_16 = perc_difference_16.astype(float)
diff = perc_difference_16-perc_difference_12
diff

array([        nan, -0.06653383, -0.04118679,  0.06154231,  0.09065405,
       -0.01602986, -0.0428437 ,  0.05965599, -0.06889856, -0.02196869,
        0.02127256, -0.08562986, -0.15948696, -0.03570099,  0.00155769,
       -0.09594675,  0.00136109, -0.08256713, -0.02814982,  0.05432321,
        0.00726051, -0.12689232, -0.09870574, -0.06251926, -0.10306611,
       -0.06835505, -0.08346208, -0.0175548 , -0.19376047, -0.0479124 ,
       -0.05474053, -0.03601187, -0.0101138 , -0.04176926, -0.04866872,
       -0.10896324, -0.05077576, -0.00628566, -0.06393449, -0.11274131,
       -0.04007624, -0.13507731, -0.06489077,  0.06549292,  0.23887209,
        0.0195334 , -0.06074703,  0.04336346, -0.07765141, -0.16928941,
       -0.08935981])

In [8]:
#Get rid of the nan value
diff = diff[~np.isnan(diff)]
dems_improved = np.count_nonzero(diff>0)
republicans_improved = np.count_nonzero(diff<0)

In [9]:
#Bar chart
data = [go.Bar(
            x=['Democrats improved', 'Republicans improved'],
            y=[dems_improved, republicans_improved]
    )]
layout = go.Layout(
    title="In how many states did the two parties improve?"
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

In [10]:
#This is very high level. Let's try to see how many states republicans actually converted compared to the 2012 election
gop_12 = perc_difference_12<0 #won by the gop in 2012
gop_16 = perc_difference_16<0 #won by the gop in 2016
lst = list(zip(gop_12, gop_16))
lst

[(False, True),
 (True, True),
 (True, True),
 (True, True),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, True),
 (True, True),
 (False, False),
 (False, True),
 (True, True),
 (False, False),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (False, False),
 (False, False),
 (False, False),
 (False, True),
 (False, False),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, False),
 (False, True),
 (True, True),
 (False, False),
 (False, True),
 (False, False),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (True, True),
 (False, False),
 (False, False),
 (False, False),
 (False, True),
 (True, True),
 (True, True)]

In [11]:
from collections import Counter
counts = dict(Counter(lst))
counts

{(False, False): 21, (False, True): 7, (True, True): 23}

In [12]:
counts["democrat_to_democrat"] = counts.pop((False, False))
counts["democrat_to_republican"] = counts.pop((False, True))
counts["republican_to_republican"] = counts.pop((True, True))
counts["republican_to_democrat"] = 0
counts

{'democrat_to_democrat': 21,
 'democrat_to_republican': 7,
 'republican_to_democrat': 0,
 'republican_to_republican': 23}

In [13]:
#Get all items from a dictionary in alphabetical order of the keys
sorted_list = sorted(counts.items(), key=lambda x: x[0])
sorted_list

[('democrat_to_democrat', 21),
 ('democrat_to_republican', 7),
 ('republican_to_democrat', 0),
 ('republican_to_republican', 23)]

In [14]:
trace0 = go.Bar(
    x=['Democrat to Democrat', 'Democrat to Republican', 'Republican to Democrat',
       'Republican to Republican'],
    y=[val for key, val in sorted_list], #I want the keys in sorted order
    marker=dict(
        color=['rgba(204,204,204,1)','rgba(204,204,204,1)', 'rgba(204,204,204,1)', 'rgba(222,45,38,0.8)']),
)

data = [trace0]
layout = go.Layout(
    title='How many states did the two parties manage to convert?',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='color-bar')

## Voting and income inequality

Now we will take a look at how income and income inequality relate to the voting patterns.

In [15]:
import csv
#we will need that dictionary to put our data in a similar format
with open('state_abbreviations.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    state_abbr_dict = dict(reader)
state_abbr_dict['district of columbia'] = "DC"

In [16]:
import pprint
with open('gini_states_data.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    gini_data = dict(reader)
    
#Put the data in the right format
for key in gini_data.keys():
    gini_data[key] = eval(gini_data[key])
    
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(gini_data)

{   'Gini': [   0.399791867,
                0.399657935,
                0.376482785,
                0.44936254600000003,
                0.438901246,
                0.53154701,
                0.7277280690000001,
                0.48247823100000004,
                0.397961587,
                0.38879489899999997,
                0.34610495,
                0.487053216,
                0.383764029,
                0.300658852,
                0.36632257700000004,
                0.42266601299999995,
                0.466088861,
                0.483965099,
                0.50558567,
                0.580388665,
                0.48904141799999995,
                0.445720136,
                0.436748475,
                0.46901077,
                0.354874611,
                0.305532992,
                0.31527397,
                0.48403284,
                0.512200654,
                0.41891861,
                0.6293997170000001,
                0.430408478,
                0

In [17]:
#replace states by state abbreaviations
states_encoded = []
for state in gini_data['State']:
    state = state.lower() #convert to lowercase
    if state in state_abbr_dict.keys():
        states_encoded.append(state_abbr_dict[state])
gini_data['State'] = states_encoded

In [18]:
gini = list(zip(gini_data['Year'], gini_data['State'], gini_data['Gini']))
#keep data from 2010
gini_2010 = [i for i in gini if i[0]==2010]
len(gini_2010)

52

In [19]:
sorted(gini_2010, key = lambda x:x[1])

[(2010, 'AK', 0.632009983),
 (2010, 'AL', 0.681595385),
 (2010, 'AR', 0.6291412120000001),
 (2010, 'AZ', 0.564963937),
 (2010, 'CA', 0.620458305),
 (2010, 'CO', 0.567936897),
 (2010, 'CT', 0.549246907),
 (2010, 'DC', 0.572081685),
 (2010, 'DE', 0.59138608),
 (2010, 'FL', 0.62091881),
 (2010, 'GA', 0.5514155629999999),
 (2010, 'HI', 0.574038565),
 (2010, 'IA', 0.607103229),
 (2010, 'ID', 0.629451573),
 (2010, 'IL', 0.5973303320000001),
 (2010, 'IN', 0.589851499),
 (2010, 'KS', 0.5894688370000001),
 (2010, 'KY', 0.6125893),
 (2010, 'LA', 0.5845420960000001),
 (2010, 'MA', 0.623285711),
 (2010, 'MD', 0.582541764),
 (2010, 'ME', 0.6818228359999999),
 (2010, 'MI', 0.5895479920000001),
 (2010, 'MN', 0.684081972),
 (2010, 'MO', 0.573903501),
 (2010, 'MS', 0.58449775),
 (2010, 'MT', 0.5598515270000001),
 (2010, 'NC', 0.595159471),
 (2010, 'ND', 0.630670428),
 (2010, 'NE', 0.592691958),
 (2010, 'NH', 0.591243804),
 (2010, 'NJ', 0.578736603),
 (2010, 'NM', 0.5760961170000001),
 (2010, 'NV', 0.58

In [20]:
gini_2010.remove((2010, 'SD', 0.655813515))
len(gini_2010)

51

In [21]:
gini_2010 = dict([(i[1], i[2]) for i in gini_2010])
gini_2010

{'AK': 0.632009983,
 'AL': 0.681595385,
 'AR': 0.6291412120000001,
 'AZ': 0.564963937,
 'CA': 0.620458305,
 'CO': 0.567936897,
 'CT': 0.549246907,
 'DC': 0.572081685,
 'DE': 0.59138608,
 'FL': 0.62091881,
 'GA': 0.5514155629999999,
 'HI': 0.574038565,
 'IA': 0.607103229,
 'ID': 0.629451573,
 'IL': 0.5973303320000001,
 'IN': 0.589851499,
 'KS': 0.5894688370000001,
 'KY': 0.6125893,
 'LA': 0.5845420960000001,
 'MA': 0.623285711,
 'MD': 0.582541764,
 'ME': 0.6818228359999999,
 'MI': 0.5895479920000001,
 'MN': 0.684081972,
 'MO': 0.573903501,
 'MS': 0.58449775,
 'MT': 0.5598515270000001,
 'NC': 0.595159471,
 'ND': 0.630670428,
 'NE': 0.592691958,
 'NH': 0.591243804,
 'NJ': 0.578736603,
 'NM': 0.5760961170000001,
 'NV': 0.582882047,
 'NY': 0.599277914,
 'OH': 0.591796398,
 'OK': 0.573493421,
 'OR': 0.590557694,
 'PA': 0.591880679,
 'RI': 0.543013096,
 'SC': 0.57542032,
 'SD': 0.626876354,
 'TN': 0.59136045,
 'TX': 0.564221501,
 'UT': 0.617894828,
 'VA': 0.666916549,
 'VT': 0.588738322,
 'WA

In [22]:
#income data
income = Table.read_table("states_median_income.csv")
income.show(5)

State,median_income
alabama,"41,459"
alaska,"66,311"
arizona,"48,108"
arkansas,"39,375"
california,"59,540"


In [23]:
#format the income column
income.append_column("median_income",[int(i.replace(",", "")) for i in income.column("median_income")])
income.append_column("State", [state_abbr_dict[key] for key in income.column("State")])

In [24]:
median_income = income.sort("State").column("median_income")
gini = [i[1] for i in sorted(list(gini_2010.items()), key=lambda x:x[0])]

In [25]:
electoral = Table.read_table("electoral_votes.csv").select(["State", "electoral_votes"]).take(np.arange(51))
electoral['State'] = electoral.apply(lambda x: state_abbr_dict[x.lower()], "State")
electoral_votes = electoral.sort("State").column("electoral_votes")

In [26]:
hover_text = []
bubble_size = []
for i in range(len(state_abbreviations)): 
    hover_text.append(('State: {state}<br>'+
                          'Electoral Votes: {evotes}<br>'+
                          'Median income: {income}<br>'+
                          'Gini coefficient: {gini}<br>').format(state=state_abbreviations[i],
                                                evotes=electoral_votes[i],
                                                income=median_income[i],
                                                gini=gini[i]
                                                ))
    bubble_size.append(electoral_votes[i])

In [27]:
democratic_states = perc_difference_16>0
republicans_states = perc_difference_16<0

In [28]:
text = hover_text
size = bubble_size

trace0 = go.Scatter(
    x=median_income[democratic_states],
    y=np.array(gini)[democratic_states],
    mode='markers',
    name='Democratic',
    text=np.array(text)[democratic_states],
    marker=dict(
        symbol='circle',
        sizemode='diameter',
        sizeref=0.85,
        size=np.array(size)[democratic_states],
        line=dict(
            width=2
        )
    )
)
trace1 = go.Scatter(
    x=median_income[republicans_states],
    y=np.array(gini)[republicans_states],
    mode='markers',
    name='Republican',
    text=np.array(text)[republicans_states],
    marker=dict(
        sizemode='diameter',
        sizeref=0.85,
        size=np.array(size)[republicans_states],
        line=dict(
            width=2
        ), color = 'rgb(243, 0, 0)'
    )
)

data = [trace0, trace1]
layout = go.Layout(
    title = "Income inequality and the 2016 election",
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='2106')

## Bonus material - Adnan

In [7]:
df[df.State == "michigan"]

,County Name,State,ObamaVotes.12,RomneyVotes.12,votes_dem.16,votes_gop.16,total_votes.16,per_dem.16,per_gop.16,diff.16,...,public_transportation_to_work,working_at_home,occupation_management_business_science_arts,income_less_than_10000,income_10000_to_24999,income_25000_to_49999,income_50000_to_99999,income_100000_to_149999,income_150000_to_199999,income_more_than_200000
38,alcona,michigan,2462.0,3571.0,1732.0,4201.0,6179.0,0.280304,0.679883,1037.0,...,7.0,313.0,809.0,363.0,1229.0,1520.0,1194.0,202.0,202.0,52.0
44,alger,michigan,2212.0,2329.0,1663.0,2585.0,4513.0,0.368491,0.572790,2648.0,...,47.0,224.0,915.0,296.0,800.0,1194.0,1062.0,237.0,237.0,47.0
46,allegan,michigan,19594.0,29023.0,17932.0,33812.0,55153.0,0.325132,0.613058,515.0,...,92.0,1946.0,13907.0,2423.0,6495.0,12028.0,15327.0,4198.0,4198.0,762.0
58,alpena,michigan,6549.0,7298.0,4877.0,9090.0,14688.0,0.332040,0.618873,1674.0,...,41.0,665.0,3941.0,1119.0,3524.0,3876.0,3528.0,977.0,977.0,118.0
77,antrim,michigan,5107.0,7917.0,4448.0,8469.0,13572.0,0.327734,0.624005,1623.0,...,62.0,654.0,2706.0,669.0,2141.0,3043.0,3008.0,807.0,807.0,192.0
86,arenac,michigan,3669.0,4054.0,2238.0,4704.0,7457.0,0.300121,0.630817,1034.0,...,69.0,325.0,1654.0,570.0,1626.0,2241.0,1780.0,318.0,318.0,59.0
133,baraga,michigan,1574.0,1866.0,1156.0,2158.0,3486.0,0.331612,0.619048,2.0,...,27.0,81.0,1037.0,326.0,620.0,1098.0,1019.0,201.0,201.0,38.0
143,barry,michigan,11496.0,16651.0,9109.0,19197.0,30265.0,0.300975,0.634297,10.0,...,14.0,762.0,7014.0,955.0,3586.0,6372.0,8510.0,2409.0,2409.0,416.0
155,bay,michigan,26797.0,23735.0,21641.0,28327.0,52932.0,0.408845,0.535158,2233.0,...,283.0,1573.0,13927.0,3528.0,7931.0,13545.0,13709.0,4022.0,4022.0,634.0
193,benzie,michigan,4683.0,5071.0,4108.0,5539.0,10221.0,0.401918,0.541923,393.0,...,34.0,473.0,2441.0,372.0,1314.0,2510.0,2419.0,547.0,547.0,94.0


In [9]:
url = 'http://catalog.civicdashboards.com/dataset/08b4daa7-8888-4e03-ad73-ae2ff3c47771/resource/b12bbaa4-14df-4293-8636-2bb80804678c/download/89e296fe1c5c410d83c2d97155366e24temp.geojson'
michigan_data = requests.get(url)
michigan_data = michigan_data.json()

{'features': [{'geometry': {'coordinates': [[[[-85.864155, 46.702662],
       [-85.864932, 46.504757999999995],
       [-86.364953, 46.505852],
       [-86.36526099999999, 46.331928999999995],
       [-86.615397, 46.333262999999995],
       [-86.615601, 46.158691],
       [-87.11608, 46.158944999999996],
       [-87.11663399999999, 46.494808],
       [-87.112245, 46.749925999999995],
       [-87.112695, 47.817997999999996],
       [-86.94062199999999, 47.749905],
       [-86.949226, 47.749905],
       [-86.75012, 47.67196],
       [-86.75012, 47.674910999999994],
       [-86.631321, 47.624919999999996],
       [-86.634344, 47.624919999999996],
       [-86.625125, 47.617767],
       [-86.625125, 47.620008],
       [-86.50013, 47.568324],
       [-86.25013899999999, 47.477903],
       [-86.25013899999999, 47.474447999999995],
       [-86.125141, 47.423468],
       [-86.125142, 47.416744],
       [-86.020032, 47.374962],
       [-86.027345, 47.374961],
       [-86.00014499999999, 47.36188

In [11]:
county_names = []
county_names_dict = {}

for county in michigan_data['features']:
    for m in range(len(county['properties']['name'])):
        if county['properties']['name'][m:m+6] == 'County':
            county_names.append(county['properties']['name'][0:m-1].lower())
            county_names_dict[county['properties']['name'][0:m-1]] = county['properties']['name']
            
print(county_names)

['alger', 'cheboygan', 'barry', 'kalamazoo', 'menominee', 'grand traverse', 'mackinac', 'kalkaska', 'ottawa', 'roscommon', 'saginaw', 'ogemaw', 'tuscola', 'mason', 'ingham', 'ontonagon', 'branch', 'montmorency', 'osceola', 'wexford', 'alcona', 'allegan', 'huron', 'hillsdale', 'dickinson', 'chippewa', 'leelanau', 'clare', 'baraga', 'alpena', 'gratiot', 'wayne', 'oscoda', 'bay', 'muskegon', 'keweenaw', 'houghton', 'cass', 'crawford', 'newaygo', 'midland', 'gladwin', 'washtenaw', 'mecosta', 'luce', 'oceana', 'jackson', 'montcalm', 'sanilac', 'shiawassee', 'eaton', 'st. joseph', 'schoolcraft', 'st. clair', 'isabella', 'benzie', 'kent', 'emmet', 'monroe', 'gogebic', 'manistee', 'antrim', 'iosco', 'lake', 'livingston', 'delta', 'van buren', 'berrien', 'missaukee', 'macomb', 'iron', 'lenawee', 'ionia', 'arenac', 'calhoun', 'clinton', 'presque isle', 'charlevoix', 'otsego', 'genesee', 'lapeer', 'marquette', 'oakland']


In [12]:
red_counties_2012 = []
blue_counties_2012 = []
df = pd.read_csv("https://raw.githubusercontent.com/adnanhemani/data8-assets/master/merged_election_data.csv").drop("Unnamed: 0", axis=1)
mi_voting_data = df[df.State == "michigan"]

In [22]:
import re

mi_voting_data.columns = [c.replace(' ', '_') for c in mi_voting_data.columns]

blue_counties_2016 = []
red_counties_2016 = []

for k, county in enumerate(county_names):
    county_cleaned = re.sub("\.", "", county)
    county_cleaned = re.sub(" ", "", county_cleaned)
    county_cleaned = re.sub(".*-", "", county_cleaned)
    row = mi_voting_data[mi_voting_data.County_Name == county_cleaned]
    if (len(row.index) != 1):
        print("Error: " + county_cleaned)
    else:
        if (row["votes_dem.16"].item() > row["votes_gop.16"].item()):
            blue_counties_2016.append(michigan_data['features'][k])
        else:
            red_counties_2016.append(michigan_data['features'][k])

    

In [23]:
red_data_2016 = {"type": "FeatureCollection"}
red_data_2016['features'] = red_counties_2016

blue_data_2016 = {"type": "FeatureCollection"}
blue_data_2016['features'] = blue_counties_2016

with open('florida-red-data-2016.json', 'w') as f:
    f.write(json.dumps(red_data_2016))
with open('florida-blue-data-2016.json', 'w') as f:
    f.write(json.dumps(blue_data_2016))


In [32]:
import plotly.plotly as py
import plotly.graph_objs as graph_objs

mapbox_access_token = 'pk.eyJ1IjoiYWRuYW5oZW1hbmkiLCJhIjoiY2l5N3RnY3M4MDAxZTJxb3Y3anlsbnJ0ZSJ9.REcsaI8vU6ylkrpkjuMtqA'

data = graph_objs.Data([
    graph_objs.Scattermapbox(
        lat=['45.5017'],
        lon=['-73.5673'],
        mode='markers',
    )
])
layout = graph_objs.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = red_counties_2016,
                type = 'fill',
                color = 'rgba(163,22,19,0.8)'
            ),
            dict(
                sourcetype = 'geojson',
                source = blue_counties_2016,
                type = 'fill',
                color = 'rgba(40,0,113,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=27.8,
            lon=-83
        ),
        pitch=0,
        zoom=5.2,
        style='light'
    ),
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='county-level-choropleths-python')


In [31]:
red_counties_2016

[{'geometry': {'coordinates': [[[[-85.864155, 46.702662],
      [-85.864932, 46.504757999999995],
      [-86.364953, 46.505852],
      [-86.36526099999999, 46.331928999999995],
      [-86.615397, 46.333262999999995],
      [-86.615601, 46.158691],
      [-87.11608, 46.158944999999996],
      [-87.11663399999999, 46.494808],
      [-87.112245, 46.749925999999995],
      [-87.112695, 47.817997999999996],
      [-86.94062199999999, 47.749905],
      [-86.949226, 47.749905],
      [-86.75012, 47.67196],
      [-86.75012, 47.674910999999994],
      [-86.631321, 47.624919999999996],
      [-86.634344, 47.624919999999996],
      [-86.625125, 47.617767],
      [-86.625125, 47.620008],
      [-86.50013, 47.568324],
      [-86.25013899999999, 47.477903],
      [-86.25013899999999, 47.474447999999995],
      [-86.125141, 47.423468],
      [-86.125142, 47.416744],
      [-86.020032, 47.374962],
      [-86.027345, 47.374961],
      [-86.00014499999999, 47.361883],
      [-86.00014399999999, 47.3672